### Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [3]:
import pandas as pd

In [26]:
df = pd.read_csv('C:/Netology/PYDA_14/09_Pandas/ml-latest-small/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [27]:
def set_rating(rating):
    if rating <= 2.0:
        return 'low'
    elif rating <= 4.0:
        return 'medium'
    else:
        return 'high'

In [30]:
df['class'] = df['rating'].apply(set_rating)
df.head()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,medium
1,1,1029,3.0,1260759179,medium
2,1,1061,3.0,1260759182,medium
3,1,1129,2.0,1260759185,low
4,1,1172,4.0,1260759205,medium


In [31]:
df['class'].value_counts().head()

medium    63801
high      22818
low       13385
Name: class, dtype: int64

### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [26]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }

In [47]:
cities = [x for l in geo_data.values() for x in l]

In [30]:
df = pd.read_csv('C:/Netology/PYDA_14/09_Pandas/ml-latest-small/keywords.csv')
df.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [48]:
def set_region(keyword):
    list_keyword = keyword.strip().split(' ')
    for key in list_keyword:
        if key in cities:
            return search_region(key)
        else:
            return 'undefined'

In [ ]:
def set_region(keyword):
    list_keyword = keyword.strip().split(' ')
    for key in list_keyword:
        if key in cities:
            return search_region(key)

    return 'undefined'

In [49]:
def search_region(city):
    for reg, cities_1 in geo_data.items():
        if city in cities_1:
            return reg

In [53]:
df['region'] = df['keyword'].apply(set_region)

In [54]:
df.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [55]:
df.region.value_counts()

undefined         99956
Центр                34
Северо-Запад          5
Дальний Восток        5
Name: region, dtype: int64

### Задание 3 (бонусное)
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [1]:
import re

In [4]:
df_1 = pd.read_csv('C:/Netology/PYDA_14/09_Pandas/ml-latest-small/movies.csv')
df_1.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
df_2 = pd.read_csv('C:/Netology/PYDA_14/09_Pandas/ml-latest-small/ratings.csv').sort_values(by='movieId')
df_2 = df_2[['movieId', 'rating']]
df_2 = df_2.groupby('movieId').mean().reset_index()
df_2.head(7)

,movieId,rating
0,1,3.872470
1,2,3.401869
2,3,3.161017
3,4,2.384615
4,5,3.267857
5,6,3.884615
6,7,3.283019


In [18]:
def match_rating(movieId):
    if movieId in df_2['movieId'].tolist():
        return df_2[(df_2['movieId'] == movieId)]['rating'].values[0]
    else:
        return 0.0

In [19]:
df_1['rating'] = df_1['movieId'].apply(match_rating)

In [20]:
df_1.head()

,movieId,title,genres,year,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,3.872470
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,3.401869
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,3.161017
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,2.384615
4,5,Father of the Bride Part II (1995),Comedy,1995,3.267857


In [7]:
years = [x for x in range(1950, 2020)]

In [8]:
def production_year(title):
    year = 1900
    year_1 = re.findall(r'(?<=.)+\d{4}', title)
    if year_1 != []:
        if int(year_1[-1]) in years:
            year = int(year_1[-1])
    return year

In [9]:
df_1['year'] = df_1['title'].apply(production_year)

In [10]:
df_1.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [22]:
df_3 = df_1[['year', 'rating']]
df_3 = df_3.groupby('year').mean().reset_index()
df_3.sort_values(by='rating', ascending=False).head(15)

,year,rating
2,1951,3.906653
13,1962,3.772962
24,1973,3.688212
4,1953,3.635020
8,1957,3.603241
23,1972,3.585295
22,1971,3.576986
9,1958,3.571956
0,1900,3.563229
28,1977,3.561309
